# analysis of inertial partcles in daily averaged field of NWES
In this notebook we analyse the effect of MR advection on the trajectory of plastics.   
The particles were released in the daily flowfield in the NWES from the NWES dataset (https://doi.org/10.48670/moi-00054)    
They are released the 1st of every month from 1 september 2023 to 1 march 2024 and simulated for 30 days, where there   
location is saved very hour.  

We analyse 3 types of particles:
1. tracer particles
2. randomly displaced tracer particles
4. SMMR particles with flexible Rep

We compare 2-3 to 1 to study effect of MR vs tracer particles

We analyse:
- trajectory length 
- seperation distance
- denstity distribution 

And then compare to the results of the simulation with daily resolution

24/8 update:
we might want to compare the SMMRG particles to the randomly displaced tracer particles to show that the difference grow as fast or much faster   
9/1 update: we drop the displaced tracer and introduce the skill score instead

In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

# import needed packages
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import Divider, Size
from matplotlib import colormaps
import matplotlib.cm as cm
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from decimal import Decimal
import hexbin_functions as hexfunc
import h3
from shapely.geometry import shape, Point
from datetime import datetime, timedelta
import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/simulations")
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from analysis_functions import  make_PDF, make_lognormal_PDF, Haversine
from analysis_functions_xr import trajectory_length, calc_tidal_av, skill_score, normalized_separation_dist
from particle_characteristics_functions import factor_drag_white1991

from helper import create_filelist

# set plotscale
plt.style.use('../python_style_Meike.mplstyle')
colorlist=np.array(['black','grey','firebrick'])#'orange','green','red','navy','pink'])
lineslist = ['-','-','--','--',':']


In [ ]:

#set directories and template filenames
base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'


basefile_MR_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}_tres_daily'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}.zarr')

basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_tres_daily'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}.zarr')


basefile_tracer = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}_tres_daily'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_{land_handling}.zarr')
basefile_tracer_random = (base_directory + 'tracer_random/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}_tres_daily'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_d{d:04d}_{land_handling}.zarr')



particle_types = ['tracer','inertial_SM_drag_Rep','inertial_SM_Rep_constant']#,'inertial_drag_Rep'] # 'tracer_random'
basefiles = {'tracer':basefile_tracer,
             'tracer_random':basefile_tracer_random,
             'inertial_SM_drag_Rep':basefile_MR_Rep_drag,
              'inertial_SM_Rep_constant':basefile_Rep_constant,
             'inertial_drag_Rep':basefile_MR_Rep_drag

             }

legend_names = {'tracer':'tracer',
             'tracer_random':'displaced tracer',
             'inertial_SM_drag_Rep':'c$_p$(t)',
             'inertial_drag_Rep':'MR'
             }



In [ ]:
# set constants 
Rearth = 6371 * 10**3 # in k,
deg2rad = np.pi / 180.

#for simulation
runtime =  timedelta(days=30)# timedelta(days=10)
loc = 'NWES'
runtime=timedelta(30)
nparticles = 51548# 88347 # 52511
land_handling = 'anti_beaching'
coriolis = True
Replist=[100,450,1000] # [0,10,100,450,1000]
starttimes = np.array([datetime(2023, 9, 1, 0, 0, 0, 0),
                       datetime(2023, 10, 1, 0, 0, 0, 0),
                       datetime(2023, 11, 1, 0, 0, 0, 0) ,
                       datetime(2023, 12, 1, 0, 0, 0, 0) ,
                       datetime(2024, 1, 1, 0, 0, 0, 0),
                       datetime(2024, 2, 1, 0, 0, 0, 0)])


chunck_time = 100
years = np.array([2023,2024])
months = np.array([1,2,3,4,5,6,7,8,9,10,11,12])
                      
#for particle
displacement = 300 #m


B = 0.68
tau = 3196 #2994.76 #2759.97
Tmax = 720

Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.2, 0.8, 3)) # Magma(np.linspace(0.2, 0.9, 5))

## Import data

In [ ]:
data ={}
for pt in particle_types:
    data[pt]={}
    for coriolis in [True]:
        data[pt][coriolis]={}
        if(pt == 'inertial_SM_Rep_constant'):
            for Rep in Replist:
                data[pt][coriolis][Rep]={}
        else:
            data[pt][coriolis][None]={}

            
for pt in particle_types:
    for coriolis in [True]:
        for starttime in starttimes:
            endtime = starttime + runtime 
            date = f'{starttime.year:04d}/{starttime.month:02d}'
            if(pt == 'inertial_SM_Rep_constant'):
                for Rep in Replist:
                    print(file)
                    file = basefiles[pt].format(loc=loc,
                                                                    y_s=starttime.year,
                                                                    m_s=starttime.month,
                                                                    d_s=starttime.day,
                                                                    y_e=endtime.year,
                                                                    m_e=endtime.month,
                                                                    d_e=endtime.day,
                                                                    B = int(B * 1000), 
                                                                    tau = int(tau ),
                                                                    land_handling = land_handling, 
                                                                    coriolis = coriolis,
                                                                    particle_type = pt,
                                                                    Rep = Rep,
                                                                    d = displacement)
                    # print(file)
                    ds = xr.open_dataset(file,
                                            engine='zarr',
                                            chunks={'trajectory':nparticles, 'obs':chunck_time},
                                            drop_variables=['B','tau','z'],
                                            decode_times=False) #,decode_cf=False)

                    data[pt][coriolis][Rep][date]= ds 
            else:
                file = basefiles[pt].format(loc=loc,
                                                                y_s=starttime.year,
                                                                m_s=starttime.month,
                                                                d_s=starttime.day,
                                                                y_e=endtime.year,
                                                                m_e=endtime.month,
                                                                d_e=endtime.day,
                                                                B = int(B * 1000), 
                                                                tau = int(tau ),
                                                                land_handling = land_handling, 
                                                                coriolis = coriolis,
                                                                Rep = None,
                                                    
                                                                particle_type = pt,
                                                                d = displacement)
                    # print(file)
                ds = xr.open_dataset(file,
                                        engine='zarr',
                                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                                        drop_variables=['B','tau','z'],
                                        decode_times=False) #,decode_cf=False)

                data[pt][coriolis][None][date]= ds 


In [ ]:
# plot selection of trajectories
Blues = colormaps['Blues']

fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1),projection=ccrs.PlateCarree())
# Take colors at regular intervals spanning the colormap.
# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.3, 0.95, 5))
colorlist=['k','cornflowerblue','blue']
markers = ['-','--','-.']
year=2023
month=9
date = f'{year:04d}/{month:02d}'
legend= []
coriolis = True

ax.coastlines()
idstart =25 #46479#46479#22225#np.random.randint(0,nparticles)#25#8813#171#141#8969#25#25 #600#100#25#6008 # np.random.randint(0,nparticles)#22#3748 #568#25680#47584#98
print(idstart)
idstep = 1
idend = idstart+1 * idstep
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')
for pt, color, marker in zip(particle_types,colorlist,markers):


        



        # ax.legend(particle_types,loc=(1,0.2))
        # pt = 'inertial_SM_Rep_constant'
        if(pt == 'inertial_SM_Rep_constant'):
                for Rep, color2 in zip(Replist,Magmalist):
                        ax.plot(data[pt][coriolis][Rep][date].lon[idstart:idend:idstep,0::1].T,
                                data[pt][coriolis][Rep][date].lat[idstart:idend:idstep,0::1].T,
                                marker,
                                color=color2);

                        legend.append('c({Rep:d})={crep:.0f}'.format(Rep=Rep,crep=factor_drag_white1991(Rep)))
        else:
                ax.plot(data[pt][coriolis][None][date].lon[idstart,0::1].T,
                data[pt][coriolis][None][date].lat[idstart,0::1].T,
                marker,
                color=color,zorder=20);
                legend.append(legend_names[pt])
ax.legend(legend,fontsize=18, handlelength = 1.5)#,loc=(1,0.))
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
        linewidth=0, color='gray', alpha=0.5, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}
# ax.set_title(simtype[pt])

# ax.set_xlim(-15,10)
# ax.set_ylim(42,62)
# ax.set_xlim(-14.6,-10.5)
# ax.set_ylim(54.4,56.1)
# if(pt == 'inertial_SM_Rep_constant'):
#         fig.tight_layout()
#         fig.savefig('../figures/poster/trajectory_ID25_rep.pdf')
# conclusion Rep = 0, 1, 10 clearly different (but themselves very similar), 100 di

ax.plot(data[pt][coriolis][1000][date].lon[idstart,0].T,
data[pt][coriolis][1000][date].lat[idstart,0].T,
'>',
color='darkgreen',zorder=20,markersize=15);
ax.set_title('daily velocity fields',fontsize=18)
fig.savefig('../figures/hourly_vs_daily/trajectory_daily_Rep.pdf')

## Along track displacement

In [ ]:
# along track distance
for pt in particle_types: # loop over all particle types
    for starttime in starttimes: # loop over all starttime
        year = starttime.year
        month = starttime.month
        date = f'{year:04d}/{month:02d}'
        if(pt == 'inertial_SM_Rep_constant'):
            for Rep in Replist:
                ds =data[pt][coriolis][Rep][date]
                traj_length = trajectory_length(ds.lon, ds.lat)
                data[pt][coriolis][Rep][date] = ds.assign(traj_length = traj_length)
               
    
        else:
            ds=data[pt][coriolis][None][date]
            traj_length = trajectory_length(ds.lon, ds.lat)
            data[pt][coriolis][None][date] = ds.assign(traj_length = traj_length)
          
            


    


In [ ]:
#  plot PDF of total trajectory length
fig,ax=plt.subplots()

Tmax = 720

legend=[]
Tmax=720
for pt, color, marker in zip(particle_types,colorlist,markers):




    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist): 
            legend.append('c({Rep:d})={crep:.0f}'.format(Rep=Rep,crep=factor_drag_white1991(Rep)))
            data_arrays = list(data[pt][coriolis][Rep].values())
            combined_data = xr.concat(data_arrays, dim="starting_days")

            average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)

            bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),401, norm=True)
            ax.plot(bins,pdf,marker,color=color2)
    else:
        legend.append(legend_names[pt])
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days")

        average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)

        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),401, norm=True)


        ax.plot(bins,pdf,marker,color=color,zorder=20)
# ax.set_title(simtype[pt])
ax.legend(legend)
ax.set_ylabel("PDF")
ax.set_xlabel("trajectory length [km]")


    #tidal_av
   


# for particle_type in particle_types:
#     bins, pdf = make_PDF(arrays[particle_type+'_tidal_av'],401, norm=True)
#     ax.plot(bins,pdf,'--',color=color_particles[particle_type])
#     legend.append(particle_type+' tidal av')



In [ ]:
# plot PDF difference trajectory length and rest of particles
i =0
fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1))
data_arrays_tracer = list(data['tracer'][coriolis][None].values())
combined_data_tracer = xr.concat(data_arrays_tracer, dim="starting_days")
legend = []
Tmax = 720
for pt, color, marker in zip(particle_types[1:],colorlist[1:],markers[1:]):





    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            
            diff = combined_data_Rep.traj_length - combined_data_tracer.traj_length
            average_data = diff.mean(dim="starting_days",skipna=True)
            bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
            ax.plot(bins,pdf,marker,color=color2, zorder = i)
            legend.append('c({Rep:d})={crep:.0f}'.format(Rep=Rep,crep=factor_drag_white1991(Rep)))
            i-=5
    else:
        # print(pt)

        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days") 
        diff = combined_data.traj_length - combined_data_tracer.traj_length
        average_data = diff.mean(dim="starting_days",skipna=True)
        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
        print(pdf)
        ax.plot(bins,pdf,marker,color=color)
        legend.append(legend_names[pt])

    # ax.set_title(simtype[pt])
    ax.legend(legend,fontsize=18,handlelength = 1.5)#,loc=(1,0.2))
    ax.set_ylabel("PDF")
    ax.set_xlabel("trajectory length difference to displaced tracer [km]")
    ax.set_title('daily velocity fields',fontsize=18)
    # ax.set_xlim(-50,50)
    # ax.set_yscale('log')


fig.savefig('../figures/hourly_vs_daily/PDF_trajectory_length_dif_daily_field_Rep.pdf')

## Relative dispersion

In [ ]:
# # EVALULATE IF YOU WANT TO REMOVE DISTANCE CALCULATION
# for starttime in starttimes:
#     print(starttime)
#     year = starttime.year
#     month = starttime.month
#     date = f'{year:04d}/{month:02d}'
#     for pt_a, pt_b in [['tracer_random','tracer'],['inertial_SM_drag_Rep','tracer']]:
#         data[pt_a][coriolis][date]=data[pt_a][coriolis][date].drop_vars('dist_'+pt_a+'_'+pt_b)

In [ ]:
# Relative dispersion


for starttime in starttimes:
    print(starttime)
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}' #'inertial_SM_drag_Rep','inertial_Rep_constant','inertial_SM_Rep_constant']
    ds_tracer = data['tracer'][coriolis][None][date]
    for pt in particle_types[1:]:
        if(pt == 'inertial_SM_Rep_constant'):
            for Rep in Replist:
                ds_Rep = data[pt][coriolis][Rep][date]
                dist= Haversine(ds_Rep['lon'],ds_Rep['lat'],
                                    ds_tracer['lon'],ds_tracer['lat'])
                data[pt][coriolis][Rep][date]=data[pt][coriolis][Rep][date].assign(dist = dist)
                data[pt][coriolis][Rep][date]=data[pt][coriolis][Rep][date].rename({'dist':'dist_to_tracer'})

            
        else:
            ds = data[pt][coriolis][None][date]
            dist= Haversine(ds['lon'],ds['lat'],
                                    ds_tracer['lon'],ds_tracer['lat'])
            data[pt][coriolis][None][date]=data[pt][coriolis][None][date].assign(dist = dist)
            data[pt][coriolis][None][date]=data[pt][coriolis][None][date].rename({'dist':'dist_to_tracer'})

            

        

In [ ]:
# plot distribution final distance between particles
# slow manifold finds bigger seperation distance (as expected because more oscillations)
tfinal = 799
markerlist=np.array(['--','-.',':','--','--','-.',':'])

i=0
Tmax=720
# cmap = plt.colormaps['magma']
# magma_colors = cmap(np.linspace(0.2, 0.9,4))
fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1))
legend=[]
for pt,color,marker in zip(particle_types[1:],colorlist[1:],markerlist[:]):   

    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            mean=combined_data_Rep['dist_to_tracer'].mean(dim='trajectory',skipna=True)
            # std =combined_data_Rep['dist_to_tracer'].std(dim='trajectory',skipna=True)
            average_mean = mean.mean(dim="starting_days",skipna=True)
            average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
            
            tlist=np.arange(0,Tmax,1)/24
        
            # ax.errorbar(tlist[:720], average_mean[:720],average_std[:720], fmt='-', color=color)
            ax.plot(tlist[:720], average_mean[:720],'-.', color=color2)
            ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color2,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
        
            legend.append('c({Rep:d})={crep:.0f}'.format(Rep=Rep,crep=factor_drag_white1991(Rep)))
    else:
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days") 
        mean=combined_data['dist_to_tracer'].mean(dim='trajectory',skipna=True)
        # std =combined_data_Rep['dist_to_tracer'].std(dim='trajectory',skipna=True)
        average_mean = mean.mean(dim="starting_days",skipna=True)
        average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
        
        tlist=np.arange(0,Tmax,1)/24
        # ax.errorbar(tlist[:720], average_mean[:720],average_std[:720], fmt='-', color=color)
        print(marker)
        ax.plot(tlist[:720], average_mean[:720],marker, color=color,zorder=20)
        ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
    
        legend.append(legend_names[pt])

ax.legend(legend,fontsize=18,handlelength=1.5)
ax.set_xlabel('t [days]')
ax.set_ylabel('relative distance to tracer [km]')
ax.set_title('daily velocity fields',fontsize=18)
fig.savefig('../figures/hourly_vs_daily/relative_distance_daily_Rep.pdf')